In [1]:
import numpy as np
import random
import matplotlib.pyplot as plt
import pandas as pd

from deap import algorithms
from deap import base
from deap import benchmarks
from deap import cma
from deap import creator
from deap import tools

In [24]:
def fk(init_theta1, init_theta2, a1, a2, armL=1.0):
    next_theta1 = init_theta1 + a1
    next_theta2 = init_theta2 + a2
    x = armL*np.cos(next_theta1) + armL*np.cos(next_theta1+next_theta2)
    y = armL*np.sin(next_theta1)+ armL*np.sin(next_theta1+next_theta2)
    pos_pred = np.array([x, y])
    return pos_pred

def evaluate(individual):
    a1 = individual[0]
    a2 = individual[1]
    pos_pred = fk(INIT_THETA1, INIT_THETA2, a1, a2)
    cost = np.linalg.norm(pos_pred-POS_GOAL)
    return cost, 

In [43]:
def solve(IND_DIM, NPOP, NGEN):
    np.random.seed(64)

    creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
    creator.create("Individual", np.ndarray, fitness=creator.FitnessMin)
    strategy = cma.Strategy(centroid=[0.0]*IND_DIM, sigma=2.0, lambda_=NPOP)  # cnetroid: "mu", sigma: "sigma"

    toolbox = base.Toolbox()
    toolbox.register("generate", strategy.generate, creator.Individual)
    toolbox.register("update", strategy.update)
    toolbox.register("evaluate", evaluate)


    halloffame = tools.HallOfFame(1, similar=np.array_equal)

    mean_fitness = []
    halloffame_array = []
    C_array = []
    centroid_array = []
    for gen in range(NGEN):
        stack_fitness = []
        # 新たな世代の個体群を生成
        population = toolbox.generate()
        # 個体群の評価
        fitnesses = toolbox.map(toolbox.evaluate, population)
        for ind, fit in zip(population, fitnesses):
            ind.fitness.values = fit
            stack_fitness.append(fit) 
        mean_fitness.append(np.mean(stack_fitness))
        
        # 個体群の評価から次世代の計算のためのパラメタ更新
        toolbox.update(population)

        # hall-of-fameの更新
        halloffame.update(population)

        halloffame_array.append(halloffame[0])
        C_array.append(strategy.C)
        centroid_array.append(strategy.centroid)
        
    df = pd.DataFrame({"mean_fitness":mean_fitness, "centroid":centroid_array, "C_array":C_array})
#     df=df.applymap(lambda x: x[0].item() if x[0]!="" else np.nan)
        
    return df

In [45]:
# INIT_THETA1 = 0.0
# INIT_THETA2 = 0.0
# POS_GOAL = np.array([[0.0, 2.0]])
# IND_DIM = 2
# NPOP = 40
# NGEN = 50

# creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
# creator.create("Individual", np.ndarray, fitness=creator.FitnessMin)
# strategy = cma.Strategy(centroid=[0.0]*IND_DIM, sigma=2.0, lambda_=NPOP)  # cnetroid: "mu", sigma: "sigma"

# toolbox = base.Toolbox()
# toolbox.register("generate", strategy.generate, creator.Individual)
# toolbox.register("update", strategy.update)
# toolbox.register("evaluate", evaluate)

# print(f"Individual dimentions: {len(toolbox.generate()[0])}")
# print(f"Number of Individuals: {len(toolbox.generate())}")
# print(f"sample evaluation score: {toolbox.evaluate(toolbox.generate()[0])}")
# print("")

In [54]:
INIT_THETA1 = 0.0
INIT_THETA2 = 0.0
POS_GOAL = np.array([[0.0, 2.0]])

solve(IND_DIM=2, NPOP=50, NGEN=20)

/home/yashima/anaconda3/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'FitnessMin' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)
/home/yashima/anaconda3/lib/python3.7/site-packages/deap/creator.py:141: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  RuntimeWarning)


,mean_fitness,centroid,C_array
0,2.443269,"[-0.11661359418903278, 0.20231563565354216]","[[1.5002151924590406, 0.05556631094637891], [0..."
1,2.331693,"[1.1661745108606825, 0.6995567232921044]","[[1.5345798566641737, 0.164933200690406], [0.1..."
2,2.327566,"[1.1692347994481886, 1.001460083659733]","[[1.2786492751959389, 0.048667446180479704], [..."
3,2.194918,"[1.2338057039075851, 0.836413041616125]","[[0.2452519782558847, -0.06893319491844252], [..."
4,1.223535,"[1.5139524002881017, 0.029440828337262036]","[[0.11330673189544505, -0.14904073378512492], ..."
5,1.096605,"[1.5108530713092605, 0.06870985501542534]","[[0.0411024896194974, -0.07271855413174219], [..."
6,0.390393,"[1.5866785590562864, -0.0023596760999305]","[[0.019116102118496528, -0.032149798588757654]..."
7,0.189063,"[1.569201355743052, 0.008982779864979591]","[[0.008975830206659345, -0.01715913875288527],..."
8,0.066224,"[1.5801311943057277, -0.018868721057780397]","[[0.0040811112769312905, -0.008104766925352086..."
9,0.027769,"[1.601793168285037, -0.060180727061959635]","[[0.0021960121056257493, -0.004374639512548857..."
